# Workforce scheduling optimization

Within the context of workforce management, this notebook showcases a key exercise to develop a workforce schedule with optimal utilization of resources.

## Problem statement

In each hour slot of a day, we have forecasted the required number of BPOs to satisfy our demand:

- 7:00 - 8:00 : 12 workers needed
- 8:00 - 9:00 : 10 workers needed
- ...

Since BPOs work in shifts (typically 4 or 8 hours), their wages also vary. Our task is to create a work schedule to meet the hourly staffing requirements while simultaneously minimizing the overall cost of labor.

## Build scheduling table

First we build a scheduling table showing:
- All possible 4-hour shifts and 8-hour shifts
- Each working hour slots in a day
- Hourly staffing requirements
- Wage per shift

![schedule_table](static/schedule_table.png)

In [2]:
from __future__ import annotations

from dataclasses import dataclass

import pandas as pd
import numpy as np

import pulp as solver

In [3]:
# Input variables

WORK_START = 7
WORK_END = 22

# Worker requirement per hour slot from WORK_START to WORK_END
STAFFING_REQUIREMENT = [
    6, 20, 24, 18, 79, 72, 60, 25, 47, 47, 49, 63, 56, 42, 34, 14
]


@dataclass
class ShiftType:
    name: str
    hours: int
    wage: int


part_time = ShiftType("Part-time", hours=4, wage=25)
full_time = ShiftType("Full-time", hours=8, wage=40)
shift_types = [part_time, full_time]


def build_scheduling_table(
        shift_types: list[ShiftType],
        staffing_requirement: list[int] = STAFFING_REQUIREMENT,
        work_start: int = WORK_START,
        work_end: int = WORK_END) -> pd.DataFrame:
    hours = range(work_start, work_end + 1)
    shifts = []
    shift_tables = []
    wages = []

    for shift_type in shift_types:
        # generate all possible shift ranges for this type of shift
        # e.g for 4-hour shift: 7-10, 8-11, ...
        possible_shifts = [
            (start_hour, start_hour + shift_type.hours - 1)
            for start_hour in range(work_start, work_end - shift_type.hours +
                                    2)
        ]
        shifts += possible_shifts

        # create a shift table, populate with 0s
        shift_table = np.zeros((len(hours), len(possible_shifts)), int)
        
        # fill the hour slot this shift covers with 1
        for hour in range(shift_type.hours):
            np.fill_diagonal(shift_table[hour:], 1)

        wage = np.ones(shift_table.shape[1]) * shift_type.wage

        shift_tables.append(shift_table)
        wages.append(wage)

    shift_arr = np.concatenate(shift_tables, axis=1)
    wage_arr = np.concatenate(wages)
    data = np.vstack((shift_arr, wage_arr))  # add wage as the final row
    column_name = [f"shift_[{start:>2} {end:>2}]" for start, end in shifts]
    index_name = list(hours) + ["wage"]  # add `wage`` as the final row index
    # add [0] to ensure the array have the same element after + wage row
    worker_needed = staffing_requirement + [0] 

    sched_table = pd.DataFrame(data, columns=column_name, index=index_name)
    sched_table["worker_needed"] = worker_needed
    return sched_table


sched_table = build_scheduling_table(shift_types)
sched_table

,shift_[ 7 10],shift_[ 8 11],shift_[ 9 12],shift_[10 13],shift_[11 14],shift_[12 15],shift_[13 16],shift_[14 17],shift_[15 18],shift_[16 19],...,shift_[ 7 14],shift_[ 8 15],shift_[ 9 16],shift_[10 17],shift_[11 18],shift_[12 19],shift_[13 20],shift_[14 21],shift_[15 22],worker_needed
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
8,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
9,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24
10,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,18
11,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,79
12,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,72
13,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,60
14,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25
15,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47
16,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47


## Formulation

**Model formulation**                            

\begin{align}
\text{Minimize}   & \sum_{j=0}^{\infty} {w_j y_j} \\
\text{subject to} & \sum_{j=0}^{\infty} {a_{jt} y_j} >= d_t \text{ with } t = 1, \ldots, T \\
                  & y_j >= 0 \text{ and integer } j = 1, \ldots, n
\end{align}

**Input parameter**

- Number of shifts ($n$)
- Number of time windows ($T$)
- Number of workers required per time window ($d_t$)
- Wage rate per shift ($w_j$)

**Decision variables**: number of workers need per work shift ($y_j$) \
**Constraints**: demand within each time window must be satisfied \
**Objective**: Minimize the total cost of wages paid to all workers

Let $a_{jt} =1$  if shift covers the time window $t (j=1,..,n; \ t=1,...,T)$ and $a_{jt} =0$ otherwise.



In [4]:
# Input parameters
n = sched_table.shape[1] - 1
T = sched_table.shape[0] - 1
d = STAFFING_REQUIREMENT
w = sched_table.loc["wage", sched_table.columns != "worker_needed"].astype(int)
a = sched_table \
  .loc[sched_table.index != "wage", sched_table.columns != "worker_needed"] \
  .astype(int).to_numpy()

print(f"""Input parameters:
    n: {n}
    T: {T}
    d: {d}
    w: {w.to_numpy()}
    a: {a}
""")

Input parameters:
    n: 22
    T: 16
    d: [6, 20, 24, 18, 79, 72, 60, 25, 47, 47, 49, 63, 56, 42, 34, 14]
    w: [25 25 25 25 25 25 25 25 25 25 25 25 25 40 40 40 40 40 40 40 40 40]
    a: [[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0]
 [0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0]
 [0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]]



In [5]:
# Create problem
prob = solver.LpProblem("scheduling_workers", solver.LpMinimize)

# Decision variables
y = solver.LpVariable.dicts("worker_needed", list(range(n)), lowBound=0, cat="Integer")

# to convert to shift name later instead of shift 1, shift 2,..., shift n
shift_name_map = dict(zip(list(range(n)), sched_table.columns))

# Objective function
prob += solver.lpSum([w[j] * y[j] for j in range(n)])

# Subject to/ add constraint
# a_tj is the number of worker assigned to shift j at a time
for t in range(T):
    prob += solver.lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]
    
prob.solve()
print("Status:", solver.LpStatus[prob.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/chaubui/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jy/g7rv1hk17rj1fv8fw6wyyjmh0000gn/T/e0e1ad0595d644919a6254fb3fe3a3a5-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/jy/g7rv1hk17rj1fv8fw6wyyjmh0000gn/T/e0e1ad0595d644919a6254fb3fe3a3a5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 21 COLUMNS
At line 212 RHS
At line 229 BOUNDS
At line 252 ENDATA
Problem MODEL has 16 rows, 22 columns and 124 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4170 - 0.00 seconds
Cgl0003I 0 fixed, 22 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 16 rows, 22 columns (22 integer (0 of which binary)) and 124 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of 4170 fo

In [6]:
solution = [y[j].value() for j in range(n)]

for i, y_ in enumerate(solution):
    if y_ > 0:
        shift_name = shift_name_map.get(i)
        print(f"{shift_name} needs {y_} workers")

print(f"The total amount of money being paid to workers are ${solver.value(prob.objective)}",)

shift_[ 8 11] needs 19.0 workers
shift_[11 14] needs 27.0 workers
shift_[16 19] needs 2.0 workers
shift_[18 21] needs 14.0 workers
shift_[19 22] needs 20.0 workers
shift_[ 7 14] needs 6.0 workers
shift_[11 18] needs 27.0 workers
shift_[12 19] needs 12.0 workers
shift_[13 20] needs 8.0 workers
The total amount of money being paid to workers are $4170.0
